Now I will perform topic modeling by using gensim

In [2]:
import nltk  # Import the nltk module

nltk.download('wordnet')  

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.utils import simple_preprocess

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package wordnet to /Users/lulu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import gensim
from gensim import corpora
from gensim.models import LdaModel

# Step 1: Read the preprocessed text from a file
# Assuming each line in the file is a separate document and words are separated by spaces
file_path = "./deepseek_output_lemmas.txt"  

# Read the file and create a list of tokenized documents (list of words)
with open(file_path, mode="r", encoding="utf-8") as file:
    text_list = [line.strip().split() for line in file.readlines()]  # Tokenize each line by space

# Step 2: Create a Dictionary
dictionary = corpora.Dictionary(text_list)

# Step 3: Create a Bag-of-Words Corpus
corpus = [dictionary.doc2bow(text) for text in text_list]

# Step 4: Train the LDA Model
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=20)

# Step 5: Print Topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

# Optional: Evaluate coherence score (optional but recommended)
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=text_list, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda.get_coherence()
print("Coherence Score: ", coherence_score)


In [ ]:
# Add irrelevant words to stop_words
additional_stopwords = {"said", "u", "company", "deepseeks", "tech","say","china","chinese"}
stop_words = set(stopwords.words('english')).union(additional_stopwords)

def preprocess(text):
    tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize
    tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation/numbers
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return tokens


In [ ]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# Create a dictionary and a corpus from the tokenized data
dictionary = Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# Print the dictionary and corpus to check if everything looks fine
print(dictionary.token2id)  # Check word-to-ID mapping
print(corpus[:2])  # Check corpus (list of word counts per document)


In [ ]:
# Train the LDA model
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=30)

# Print the topics
topics = lda_model.print_topics(num_words=5)  # Print the top 5 words for each topic
for topic in topics:
    print(topic)


In [ ]:
coherence_model = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass', dictionary=dictionary)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score (u_mass): {coherence_score}")


In [ ]:
# Calculate coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df['tokens'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f'Coherence Score: {coherence_lda}')


In [ ]:
#I want to remove the word deepseek ,ai,china,since they are meaningless, what do you think? I acuually expect topic like "tech""politic""economy"words like "deepseek," "ai," and "china" may dominate the topics and make them overly specific to certain aspects of your dataset, especially if they don't align with the broader themes you're aiming for, like tech, politics, and economy.
#If you're looking to obtain more general topics like tech, politics, and economy, removing these highly frequent and specific words can definitely help the model focus on more meaningful, diverse, and interpretable topics.
#Modify the Preprocessing to Exclude Specific Words:
#To ensure words like "deepseek," "ai," and "china" are excluded from the topics, you can add them to a custom stopwords list and remove them during the preprocessing stage.
custom_stopwords = stop_words.union({"deepseek", "ai", "china", "chinese", "openai"})

def preprocess(text):
    tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize
    tokens = [word for word in tokens if word.isalpha()]  # Remove punctuation/numbers
    tokens = [word for word in tokens if word not in custom_stopwords]  # Remove custom stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatization
    return tokens
